# Introduction

# Gathering Data

Data is collected from 3 different sources:

twitter_archive_enhanced.csv = WeRateDogs twitter account archive, which is available in csv-format and will be downloaded manually.

image_predictions.tsv = Predictions what breed of dog is present in each tweet. This file will be downloaded programmatically by using the Requests library.

tweet_json.txt = Entire tweets from WeRateDogs Twitter archive. This file will be created  programmatically from WeRateDogs Twitter archive by using Twitter API  

In [ ]:
import pandas as pd

## twitter_archive_enhanced.csv 

In [ ]:
df_archive = pd.read_csv('project_wrangle_and_analyse_data/twitter-archive-enhanced.csv')
df_archive.head()

## image-predictions.tsv 

In [ ]:
from bs4 import BeautifulSoup
import requests 
import os
import pandas as pd

In [ ]:
folder_name = 'project_wrangle_and_analyse_data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [ ]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

In [ ]:
response = requests.get(url)

In [ ]:
response

In [ ]:
with open (os.path.join(folder_name, url.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)

In [ ]:
df = pd.read_csv('project_wrangle_and_analyse_data/image-predictions.tsv', sep ='\t')

In [ ]:
df.head()

# tweet_json.txt 

In [ ]:
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = 'HIDDEN'
consumer_secret ='HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv

# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API

tweet_ids = df_archive.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)


In [ ]:
import json
tweet_data = []

with open('tweet_json.txt','r') as json_file:  
    for line in json_file:
        json_data = json.loads(line)
        tweet_id = json_data['id']
        retweet_count = json_data['retweet_count']
        favorite_count = json_data['favorite_count']
        tweet_data.append({'retweet_count': int(retweet_count),
                        'favorite_count': int(favorite_count),
                        'tweet_id': int(tweet_id)})



df = pd.DataFrame(tweet_data, columns = ['tweet_id', 'retweet_count', 'favorite_count'])

In [ ]:
df

# Assessing Data 

In [ ]:
df.info()

# Cleaning Data

# Storing and Acting on Wrangled Data